In [1]:
import os
import pandas as pd
import datetime
import time
import numpy as np

In [2]:
pd.set_option('display.max_rows',500)
data_file_name = 'line_oa_chat_csv_230717_091224'

In [3]:
os.chdir('C:\\Users\\User\\Desktop\\aigo 2023\\第二階段實作\\出題單位提供資料')
os.getcwd()

'C:\\Users\\User\\Desktop\\aigo 2023\\第二階段實作\\出題單位提供資料'

# 讀取所有對話資料及整理

In [6]:
start = time.time()

# 讀入所有資料、合併為一個檔案，並設立檔案的INDEX
col_name = ['傳送者類型','傳送者名稱','傳送日期','傳送時間','內容']
final_col = ['ttl_index','csv_num','date_time','week_day','diff_hour','dialogue_num','sentence_num'] + col_name

week_list = ["星期一","星期二","星期三","星期四","星期五","星期六","星期日"]
all_df = pd.DataFrame(columns = final_col)
for i in os.listdir(os.getcwd() + '\\' + data_file_name):
    tmp_df = pd.read_csv(data_file_name + '\\' + i ,names = col_name,encoding='utf-8',engine='python').iloc[4:,:]
    
    # 新增檔案index欄位
    tmp_df['csv_num'] = [i.split('_')[0]] * tmp_df.shape[0]
    
    # 新增對話時間的星期欄位
    tmp_df['week_day'] = tmp_df[['傳送日期']].applymap(lambda x :week_list[datetime.date(int(x.split('/')[0]),int(x.split('/')[1]), int(x.split('/')[2])).weekday()])
    
    # 合併日期與時間
    tmp_df['date_time'] = tmp_df['傳送日期'] + ' ' + tmp_df['傳送時間']
    tmp_df['date_time'] = tmp_df[['date_time']].applymap(lambda x : datetime.datetime.strptime(x.replace('/','-'), '%Y-%m-%d %H:%M:%S'))
    
    # 計算該發話與前一個發話的時間差距
    tmp_df1 = tmp_df.copy()
    tmp_df1.index = [i+1 for i in list(tmp_df.index)]
    tmp_df1.columns = ['傳送者類型', '傳送者名稱', '傳送日期', '傳送時間', '內容', 'csv_index', 'week_day', 'date_time_lasttime']
    tmp_df = pd.concat([tmp_df, tmp_df1[['date_time_lasttime']]], axis=1).iloc[:-1,:]
    tmp_df['diff_time'] = (tmp_df['date_time'] - tmp_df['date_time_lasttime']).fillna(0)
    tmp_df['diff_hour'] = tmp_df.diff_time.map(lambda x : round(x.total_seconds()/3600),2)
    
    # 新增同客戶中對話數的判斷(暫定超過24小時即為新對話)，及新增同一個對話中的流水號
    dialogue_index = list([0])
    sentence_index = list([0])
    for i in tmp_df.diff_hour.tolist()[1:]:
        if i > 24:
            dialogue_index.append(dialogue_index[-1] + 1)
            sentence_index.append(0)
        else:
            dialogue_index.append(dialogue_index[-1])
            sentence_index.append(sentence_index[-1]+1)
    tmp_df['dialogue_num'] = dialogue_index
    tmp_df['sentence_num'] = sentence_index
    
    # 產出對話最終的index
    tmp_df['ttl_index'] = tmp_df['csv_num'].astype(str) + '_' + tmp_df['dialogue_num'].astype(str)
    
    # 選取需要的欄位
    tmp_df = tmp_df[final_col]
    
    # 合併資料
    all_df = all_df.append(tmp_df).reset_index(drop=True)
    

all_df = all_df[all_df.ttl_index != 'nan_0'].reset_index(drop=True)
print('讀入檔案數:',len(set(all_df.csv_num)))

end = time.time()

# 輸出結果
print("執行時間：%f 分鐘" % ((end - start)/60))

讀入檔案數: 335
執行時間：1.176472 分鐘


In [7]:
# user_df[['內容','ttl_index']].groupby('內容').count().sort_values(by=['ttl_index'], ascending=False)

In [8]:
no_sentence = ['照片已傳送','貼圖已傳送','謝謝','你好','您好','感謝','謝謝您','對','好','好的','沒有']
user_df = all_df[(all_df.傳送者類型 == 'User') & ~(all_df.內容.isin(no_sentence))].reset_index(drop=True)

In [9]:
user_df.shape

(2597, 12)

# 餘弦相似度

In [10]:
import jieba
import jieba.analyse as analyse
import math
from string import digits

In [11]:
jieba.load_userdict("define_word.txt")
analyse.set_stop_words("stop_word.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 1.602 seconds.
Prefix dict has been built succesfully.


##### 讀入出題單位回饋的訓練資料

In [12]:
qa_df = pd.read_excel('附件三、資料格式_問題類別歸類_微調.xlsx',sheet_name='【問題類別歸類】填寫_微調')[['問題類別','客戶詢問的問題']]
qa_group_df = pd.DataFrame(qa_df.groupby('問題類別')['客戶詢問的問題'].apply(lambda x: '。'.join(x))).reset_index()

##### 找出停用詞

In [13]:
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

In [14]:
# 去除停用詞
stopwords = stopwordslist("stop_word.txt")
def del_stopword(word_list):
    new_word_list = [i for i in word_list if i not in stopwords+['\n','\t','\r'] ]
    return new_word_list

#### 問題類別定義

In [15]:
qa_define = qa_group_df.問題類別.tolist()

#### 餘弦相似度方法一

In [16]:
start = time.time()

index_num = 0
all_similarity = {}
table = str.maketrans('', '', digits)
for s1 in user_df['內容']:
    s1 = s1.translate(table) # 去除數字
    s1_cut = [i for i in jieba.cut(s1, cut_all=False) if i != '']
    s1_cut = del_stopword(s1_cut)
    
    similarity_score = []
    for s2 in qa_group_df['客戶詢問的問題']:
        s2_cut = [i for i in jieba.cut(s2, cut_all=False) if i != '']
        s2_cut = del_stopword(s2_cut)
        word_set = set(s1_cut).union(set(s2_cut))
        word_dict = dict()
        i = 0
        for word in word_set:
            word_dict[word] = i
            i += 1
        # print(word_dict)
        
        s1_cut_code = [word_dict[word] for word in s1_cut]
        # print(s1_cut_code)
        s1_cut_code = [0]*len(word_dict)
        
        for word in s1_cut:
            s1_cut_code[word_dict[word]]+=1
        # print(s1_cut_code)
        
        s2_cut_code = [word_dict[word] for word in s2_cut]
        # print(s2_cut_code)
        s2_cut_code = [0]*len(word_dict)
        for word in s2_cut:
            s2_cut_code[word_dict[word]]+=1
        # print(s2_cut_code)
        
        # 計算餘弦相似度
        sum = 0
        sq1 = 0
        sq2 = 0
        for i in range(len(s1_cut_code)):
            sum += s1_cut_code[i] * s2_cut_code[i]
            sq1 += pow(s1_cut_code[i], 2)
            sq2 += pow(s2_cut_code[i], 2)

        try:
            result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
        except ZeroDivisionError:
            result = 0.0
        #print(s1,s1_cut,s2,s2_cut,result)
        
        similarity_score.append(result)
    
    # 找出相似度最高的問題類別
    if max(similarity_score) >= 0.25:
        similarity_score.append(qa_define[similarity_score.index(max(similarity_score))])
    else:
        similarity_score.append('')
    
    similarity_score.append(s1_cut)
    
    all_similarity[index_num] = similarity_score
    index_num += 1

end = time.time()

# 輸出結果
print("執行時間：%f 分鐘" % ((end - start)/60))

執行時間：10.635886 分鐘


In [17]:
similarity_df1 = pd.DataFrame(all_similarity).T
similarity_df1.columns = qa_define + ['高相似度類別1','斷詞']

In [18]:
final_similarity_df1 = pd.concat([user_df[['ttl_index','內容']], similarity_df1], axis=1)

#### 餘弦相似度方法二

In [20]:
start = time.time()

index_num = 0
all_similarity = {}
table = str.maketrans('', '', digits)
for s1 in user_df['內容']:
    s1 = s1.translate(table) # 去除數字
    s1_cut = [i for i in jieba.cut(s1, cut_all=False) if i != '']
    s1_cut = del_stopword(s1_cut)
    
    similarity_score = []
    for qa_tag in qa_define:
        similarity_part = []
        for s2 in qa_df[qa_df.問題類別 == qa_tag].客戶詢問的問題.tolist():
            s2_cut = [i for i in jieba.cut(s2, cut_all=False) if i != '']
            s2_cut = del_stopword(s2_cut)
            word_set = set(s1_cut).union(set(s2_cut))
            word_dict = dict()
            i = 0
            for word in word_set:
                word_dict[word] = i
                i += 1
            # print(word_dict)

            s1_cut_code = [word_dict[word] for word in s1_cut]
            # print(s1_cut_code)
            s1_cut_code = [0]*len(word_dict)

            for word in s1_cut:
                s1_cut_code[word_dict[word]]+=1
            # print(s1_cut_code)

            s2_cut_code = [word_dict[word] for word in s2_cut]
            # print(s2_cut_code)
            s2_cut_code = [0]*len(word_dict)
            for word in s2_cut:
                s2_cut_code[word_dict[word]]+=1
            # print(s2_cut_code)

            # 計算餘弦相似度
            sum = 0
            sq1 = 0
            sq2 = 0
            for i in range(len(s1_cut_code)):
                sum += s1_cut_code[i] * s2_cut_code[i]
                sq1 += pow(s1_cut_code[i], 2)
                sq2 += pow(s2_cut_code[i], 2)

            try:
                result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
            except ZeroDivisionError:
                result = 0.0
            #print(s1,s1_cut,s2,s2_cut,result)

            similarity_part.append(result)
        
        # 找出QA中，相似度最高的數值，為該QA最終相似度
        similarity_score.append(max(similarity_part))
        
    # 找出相似度最高的問題類別
    if max(similarity_score) >= 0.45:
        similarity_score.append(qa_define[similarity_score.index(max(similarity_score))])
    else:
        similarity_score.append('')

    similarity_score.append(s1_cut)

    all_similarity[index_num] = similarity_score
    index_num += 1
    
end = time.time()

# 輸出結果
print("執行時間：%f 分鐘" % ((end - start)/60))

執行時間：11.302851 分鐘


In [21]:
similarity_df2 = pd.DataFrame(all_similarity).T
similarity_df2.columns = qa_define + ['高相似度類別2','斷詞']

In [22]:
final_similarity_df2 = pd.concat([user_df[['ttl_index','內容']], similarity_df2], axis=1)

# 匯出資料

In [24]:
similarity_concat = pd.concat([final_similarity_df1.iloc[:,:-1], final_similarity_df2.iloc[:,-2:]], axis=1)
similarity_concat['tag'] = np.where(((similarity_concat['高相似度類別1'] == '') & (similarity_concat['高相似度類別2'] == '')), '皆判斷不到'
         ,np.where(((similarity_concat['高相似度類別1'] != '') & (similarity_concat['高相似度類別2'] == '')), '相似度1'
         ,np.where(((similarity_concat['高相似度類別2'] != '') & (similarity_concat['高相似度類別1'] == '')), '相似度2'
         ,np.where(((similarity_concat['高相似度類別2'] == similarity_concat['高相似度類別1'])), '相同'
         , '需確認'))))

In [25]:
with pd.ExcelWriter('final_similarity_output.xlsx') as writer:  
    final_similarity_df1.to_excel(writer,index=False , sheet_name='similarity1')
    final_similarity_df2.to_excel(writer,index=False , sheet_name='similarity2')
    similarity_concat.to_excel(writer,index=False , sheet_name='similarity_merge')